<a href="https://colab.research.google.com/github/Sirfowahid/Transformer/blob/main/01_Sentiment_Analysis_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00


In [3]:
from transformers import pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split

In [4]:
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
classifier('you are great')

[{'label': 'POSITIVE', 'score': 0.9998708963394165}]

In [6]:
classifier('this is really bad')

[{'label': 'NEGATIVE', 'score': 0.9997958540916443}]

In [7]:
classifier('I can not say she is a bad girl')

[{'label': 'POSITIVE', 'score': 0.9977930784225464}]

In [8]:
classifier([
    'I love chocolate',
    'I hate myself',
    'Insted of helping me, this asshole help a girl'
])

[{'label': 'POSITIVE', 'score': 0.999846339225769},
 {'label': 'NEGATIVE', 'score': 0.9995051622390747},
 {'label': 'NEGATIVE', 'score': 0.998124897480011}]

In [9]:
import torch

In [10]:
torch.cuda.is_available()

True

In [12]:
torch.cuda.current_device()

0

In [13]:
classifier = pipeline('sentiment-analysis',device=0)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
